In [1]:
# general imports 
import tensorflow as tf 


In [2]:
# importing keras dataset 
from keras.datasets import cifar10 # standard cifar-10 plain dataset, not pre-scaled

Using TensorFlow backend.


In [3]:
# unpack train and test set 
train_set , test_set = cifar10.load_data() 
x_train, y_train = train_set 
x_test, y_test = test_set 

In [4]:
from tensorflow.contrib.data import Dataset, Iterator

# first the data which has to be processed must be converted in a tf dataset object 
training_data = Dataset.from_tensor_slices((x_train,y_train))

Instructions for updating:
Use `tf.data.Dataset.from_tensor_slices()`.


In [5]:
# zca whitening in tensorflow 
def tfZCA_fit(image, epsilon):
    flat = tf.reshape(image, (-1,32*32*3))
    sigma = tf.reduce_sum(tf.multiply(flat, flat), 1, keep_dims=True) / tf.cast(flat.shape[0],'float32')
    u, s, _ = tf.svd(sigma)
    s_inv = tf.pow(tf.sqrt(s[tf.newaxis] + epsilon), -1)
    principal_components = tf.reduce_sum(tf.multiply(tf.multiply(u,s_inv), u), 1, keep_dims=True)
    return principal_components

def tfZCA_compute(image, principal_components):
    flatx = tf.reshape(image, (-1, 32*32*3))
    whitex = tf.reduce_sum(tf.multiply(flatx, tf.transpose(principal_components)), 1, keep_dims=True)
    x = tf.reshape(whitex, [-1,32,32,3])
    return whitex
    
    

In [6]:
# using Dataset 
# An iterator object will be created to GET BATCHES OF IMAGES AND NOT SINGLE IMAGES 
batch_size = 10000

batch = training_data.batch(batch_size)

iterator = batch.make_one_shot_iterator()

next_element = iterator.get_next()


training_init_op = iterator.make_initializer(batch)

princ_comp = tfZCA_fit(tf.cast(x_train,'float32'), 1e5)


#resized = tf.cast(next_element[0],'float32')
#resized = tf.image.resize_nearest_neighbor(tf.cast(next_element[0],'float32'), [20,20]) 
#resized = tf.map_fn(lambda frame: tf.image.resize_nearest_neighbor(frame, [28,28]) , tf.cast(next_element[0],'float32') )
#global_contrast_norm = tf.map_fn(lambda frame: tf.image.per_image_standardization(frame), resized)
#rand_flipped = tf.map_fn(lambda frame: tf.image.random_flip_left_right(frame), global_contrast_norm)
#whitened = tf.map_fn(lambda frame:tfZCA(frame, 1e5), rand_flipped)
whitened = tfZCA_compute(tf.cast(next_element[0],'float32'), princ_comp)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [7]:
import time 


In [8]:
t0 = time.time()

with tf.Session() as sess:

    # initialize the iterator on the training data
    sess.run(training_init_op)

    # get each element of the training dataset until the end is reached 
    while True:
        try:
            std_image=sess.run(whitened)
            # processing goes here 
            
            
            
            
        except tf.errors.OutOfRangeError:
            print("End of training dataset.")
            break

print('Total time employed to process dataset: %.5f seconds' % (time.time() - t0))        

End of training dataset.
Total time employed to process dataset: 10.80191 seconds


In [9]:
std_image.shape


(1, 1, 3072)

In [10]:
std_image = std_image.reshape(1000,20,20,3)

ValueError: cannot reshape array of size 3072 into shape (1000,20,20,3)

In [ ]:
from images_utils import plot_images
plot_images(std_image[0:9], y_train[0:9])